In [ ]:
# default_exp followers

In [ ]:
%%capture
#test_flag_colab
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/4QFTmRoY1yy4COvnXYEDlIibcdf58LHdjLK4jrzuArEQ3L1YtAF-NrA


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
#hide
!pip install -qU nbdev fastcore
from pathlib import Path
path = Path("/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers")
%cd "{path}"
from nbdev import *
from nbdev.showdoc import *
from fastcore.test import *

     |████████████████████████████████| 61kB 1.8MB/s 
     |████████████████████████████████| 40kB 3.5MB/s 
/content/drive/My Drive/upwork/Martin followers/tomorrowswig_followers


# Instagram followers
> Here I'll create a script to fetch new followers for an instagram account. The reason is to track followers change by country, facebook API offers only lifetime numbers which are updated daily.

https://developers.facebook.com/tools/explorer/

https://developers.facebook.com/docs/facebook-login/access-tokens/

## Facebook Auth

In [ ]:
#hide
!pip install -qU gspread facebook_business python-facebook-api

     |████████████████████████████████| 1.0MB 6.0MB/s 
     |████████████████████████████████| 51kB 4.2MB/s 
     |████████████████████████████████| 10.1MB 17.0MB/s 
     |████████████████████████████████| 1.2MB 45.6MB/s 
     |████████████████████████████████| 153kB 40.4MB/s 
     |████████████████████████████████| 266kB 50.5MB/s 
ERROR: responses 0.11.0 has requirement urllib3>=1.25.10, but you'll have urllib3 1.24.3 which is incompatible.
ERROR: python-facebook-api 0.7.1 has requirement requests<3.0.0,>=2.24.0, but you'll have requests 2.23.0 which is incompatible.


In [ ]:
%%capture
#hide

import yaml
with open("facebook_creds.yml") as f:
    data = yaml.safe_load(f)
    %env APP_ID {data.get("APP_ID")}
    %env APP_SECRET {data.get("APP_SECRET")}
    %env TOKEN {data.get("TOKEN")}
    %env USER_ID {data.get("USER_ID")}

In [ ]:
#export
import os
from datetime import datetime, timedelta
from typing import *

import pandas as pd
import numpy as np
from pyfacebook import IgProApi
from pyfacebook.error import *
from tomorrowswig_followers.core import *
from tomorrowswig_followers.ads import *

In [ ]:
APP_ID = os.environ.get("APP_ID")
APP_SECRET = os.environ.get("APP_SECRET")
TOKEN = os.environ.get("TOKEN")
USER_ID = os.environ.get("USER_ID")

## Get new followers with graph api

In [ ]:
#hide
!curl -i -X GET \
 "https://graph.facebook.com/v7.0/17841409206135869/insights?metric=audience_country&period=lifetime&access_token={TOKEN}"

HTTP/2 200 
vary: Accept-Encoding
etag: "cb398e13d94bc39ba4c9d76605ff17f52e69b2f5"
x-business-use-case-usage: {"17841409206135869":[{"type":"instagram","call_count":1,"total_cputime":1,"total_time":1,"estimated_time_to_regain_access":0}]}
content-type: application/json; charset=UTF-8
facebook-api-version: v7.0
strict-transport-security: max-age=15552000; preload
pragma: no-cache
x-fb-rev: 1002702600
access-control-allow-origin: *
cache-control: private, no-cache, no-store, must-revalidate
x-fb-trace-id: A3yaTFVrhuP
x-fb-request-id: A9ISGTv4GBxiNT7YJzc_ORu
expires: Sat, 01 Jan 2000 00:00:00 GMT
x-fb-debug: J8SzFvLXR1VGGKj5L+dn/15AZW8Pz8Cktrmv6fZMOCeAZZlUMGpNkrunxKNmPfvnKRZCrjTH0yoDX2w/Okl3ag==
date: Tue, 22 Sep 2020 14:57:07 GMT
alt-svc: h3-29=":443"; ma=3600,h3-27=":443"; ma=3600

{"data":[{"name":"audience_country","period":"lifetime","values":[{"value":{"RS":57,"DE":1054,"PR":38,"BD":52,"TW":122,"PT":156,"JO":39,"JP":75,"UA":40,"FR":55,"HU":100,"BR":2599,"MA":83,"SG":759,"DZ":103,"KE

In [ ]:
api = IgProApi(app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN,
               version="5.0")

In [ ]:
try:
    response = api.get_user_insights(user_id=USER_ID, period="lifetime", metrics=["audience_country"],
                                    return_json=True)
except PyFacebookException as e:
    display(e.message)

In [ ]:
response = response[0]["values"][0]

In [ ]:
response["end_time"]

'2020-09-22T07:00:00+0000'

In [ ]:
#export
def get_followers() -> Tuple[str, Dict[str, int]]:
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
            user_id=USER_ID,
            period="lifetime",
            metrics=["audience_country"],
            return_json=True,
        )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, {0: 0})

In [ ]:
#export
def get_new_followers(date: str) -> Tuple[str, Dict[str, int]]:
    year, month, d = [int(i) for i in date.split("-")]
    api = IgProApi(
        app_id=APP_ID, app_secret=APP_SECRET, long_term_token=TOKEN, version="5.0"
    )
    try:
        response = api.get_user_insights(
                user_id=USER_ID,
                period="day",
                metrics=["follower_count"],
                since=datetime(year,month,d,0,0).strftime('%s'),
                until=datetime(year,month,d,23,59).strftime('%s'),
                return_json=True,
            )[0]["values"][0]
        return response["end_time"].split("T")[0], response["value"]
    except PyFacebookException as e:
        return (e.message, 0)

In [ ]:
get_new_followers("2020-09-17")

('2020-09-17', 37)

## Google Sheets

### Insert new followers to a google sheet 

In [ ]:
end_time, followers = get_followers()

In [ ]:
df = get_df("History")

In [ ]:
new_followers = pd.Series(followers)
date = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime(f"%b %d %Y{' '*16}")
new_followers.name = f"{date} {str(datetime.utcnow()).split('.')[0]}"
last_entry = df.iloc[:, 0]

In [ ]:
if np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values):
    # return pd.DataFrame()
    pass
else:
    df = pd.concat([df, new_followers], axis=1)
    df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
    df.index.name = "countries" 
    df.sort_index(inplace=True)

In [ ]:
df = df.fillna(0).astype(int)

In [ ]:
#export
def get_dif(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    return change_df


def get_followers_change(history_df: pd.DataFrame, date: str) -> pd.DataFrame:
    new_followers = get_dif(history_df)
    if len(date.split()) == 3:
        mask = history_df.columns.str.startswith(date)
    else:
        mask = history_df.columns == (" ".join(date.split()[:2]))
    new_followers = new_followers.iloc[:, mask].iloc[:, :1].replace(0, np.nan)
    new_followers = new_followers.dropna(axis=1, how="all").iloc[:, :1].dropna().astype(int)
    return new_followers

In [ ]:
#export
def get_ads_status(date: str):
    if get_df(date).empty:
        return "OFF"
    return "ON"

In [ ]:
history_df = get_df("History")

In [ ]:
ads_status_df = get_df("Ads Status History")
date = history_df.columns[0].split("  ")[0]
ads_status = get_ads_status(date)
change = get_followers_change(history_df, date).sum().item()

In [ ]:
last_status = pd.Series([ads_status, change, ""], name="", dtype=str, index=["Ads Status", "Change", "Followers"])
last_status.name = date
last_status

Ads Status    OFF
Change        -13
Followers        
Name: Sep 20 2020, dtype: object

In [ ]:
ads_status_df

,Sep 20 2020,Sep 19 2020,Sep 18 2020,Sep 17 2020,Sep 16 2020,Sep 15 2020,Sep 14 2020,Sep 13 2020
,,,,,,,,
Ads Status,OFF,OFF,ON,ON,OFF,OFF,OFF,OFF
Change,-13,-19,-17,30,-12,-34,-42,-30


In [ ]:
pd.concat([last_status, ads_status_df], axis=1)

,Sep 20 2020,Sep 20 2020,Sep 19 2020,Sep 18 2020,Sep 17 2020,Sep 16 2020,Sep 15 2020,Sep 14 2020,Sep 13 2020
Ads Status,OFF,OFF,OFF,ON,ON,OFF,OFF,OFF,OFF
Change,-13,-13,-19,-17,30,-12,-34,-42,-30
Followers,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#export
def save_ads_status(history_df: pd.DataFrame):
    worksheet = "Ads Status History"
    ads_status_df = get_df(worksheet)
    date = history_df.columns[0].split("  ")[0]
    ads_status = get_ads_status(date)
    change = get_followers_change(history_df, date).sum().item()
    last_status = pd.Series([ads_status, change], name="", dtype=str, index=["Ads Status", "Change"])
    last_status.name = date
    write_df(pd.concat([last_status, ads_status_df], axis=1), worksheet)

In [ ]:
# save_ads_status(get_df("History"))

In [ ]:
#export
def get_updated_followers(
    df: pd.DataFrame, data: Dict[str, int], end_time: str
) -> pd.DataFrame:
    new_followers = pd.Series(data)
    date = (datetime.strptime(end_time, "%Y-%m-%d") - timedelta(days=1)).strftime(f"%b %d %Y{' '*16}")
    new_followers.name = f"{date} {str(datetime.utcnow()).split('.')[0]}"
    last_entry = df.iloc[:, 0]
    second_last_entry = df.iloc[:, 1]
    if np.array_equal(last_entry[last_entry != 0].values, new_followers.sort_index().values) or np.array_equal(second_last_entry[second_last_entry != 0].values, new_followers.sort_index().values):
        return pd.DataFrame()
    else:
        df = pd.concat([df, new_followers], axis=1)
        df = df[[df.columns[-1]] + df.columns[:-1].tolist()]
        df.index.name = "countries" 
        df.sort_index(inplace=True)
        return df.fillna(0).astype(int)

In [ ]:
get_updated_followers(history_df, get_followers()[1], get_followers()[0])

""


In [ ]:
fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"
old_followers = {end_time: {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0},
                 "2020-07-20": {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 1}}
followers_df = pd.DataFrame.from_records(old_followers)

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)

test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
expected = {'2020-01-01': {'AR': 1001,
                            'AU': 724,
                            'BD': 55,
                            'BR': 2250,
                            'CA': 1680,
                            'CL': 526,
                            'CO': 6},
 '2020-07-20': {'AR': 100,
                'AU': 70,
                'BD': 0,
                'BR': 0,
                'CA': 0,
                'CL': 0,
                'CO': 1},
 '2020-07-21': {'AR': 100,
                'AU': 70,
                'BD': 0,
                'BR': 0,
                'CA': 0,
                'CL': 0,
                'CO': 0}}
assert test_followers_df.to_dict() == expected

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)
test_followers_df.rename(columns={test_followers_df.columns[0]: "2020-01-01" }, inplace=True)
assert test_followers_df.to_dict() == expected

In [ ]:
# test the fix for when API returns an old result shortly after it returned an updated one

fetched_followers = {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}
end_time = "2020-07-21"
old_followers = {"2020-07-21": {'AR': 100, 'AU': 70, 'BD': 0, 'BR': 0, 'CA': 0, 'CL': 0, 'CO': 0},
                 "2020-07-20": {'AR': 1001, 'AU': 724, 'BD': 55, 'BR': 2250, 'CA': 1680, 'CL': 526, 'CO': 6}}
followers_df = pd.DataFrame.from_records(old_followers)

test_followers_df = get_updated_followers(followers_df, fetched_followers, end_time)
assert test_followers_df.empty

random followers + followers from ads - unfollowers = followers gain

unfollowers = random followers + followers from ads - followers gain = new followers - followers gain

random_followers = new followers - followers from ads

In [ ]:
#export
def more_stats(df: pd.DataFrame, followers_change: pd.DataFrame, date: str) -> pd.DataFrame:
    stats = pd.Series(dtype=float, name="Followers")
    ads_followers_gain = df["Ads Followers Change"].sum()
    followers_gain = followers_change.sum().item()
    spent = df.loc["total", "Amount Spent (USD)"]
    stats["Ads Followers Change"] = ads_followers_gain
    new_followers = get_new_followers(date)[1]
    stats["Random Followers"] = new_followers - ads_followers_gain
    stats["New Followers"] = new_followers
    stats["Unfollowers"] = stats["New Followers"] - followers_gain
    stats["Followers Change"] = ads_followers_gain + stats["Random Followers"] - stats["Unfollowers"]

    stats_df = pd.DataFrame(stats)
    stats_df.index.name = "  "
    stats_df["Conversion Rate"] = stats_df["Followers"] / df.loc["total", "Clicks (All)"]
    stats_df[" "] = 0
    stats_df["Cost Per Follower/Increase"] = spent/ stats_df["Followers"]
    stats_df.iloc[1:, 1] = 0
    stats_df.iloc[[1,3,4], 3] = 0
    return stats_df.round(2)

In [ ]:
date = "Aug 13 2020"
insights_df = get_df(date)
history_df = get_df("History")

In [ ]:
followers_change = get_followers_change(history_df, date)
df = get_df(date)
if "PERFORMANCE:" in df.index:
    df = df.iloc[:df.index.get_loc("PERFORMANCE:") - 9, :]

In [ ]:
add_followers(df, followers_change)

In [ ]:
stats_df = more_stats(df, followers_change, datetime.strptime(date, "%b %d %Y").strftime("%Y-%m-%d"))

In [ ]:
get_followers_change(history_df, date)

,Aug 13 2020 2020-08-14 19:40:33
countries,
AR,-1
AU,-2
BD,1
BR,-4
CA,3
DE,4
GB,-6
ID,24
IN,6


In [ ]:
#export
def update_insights(history_df: pd.DataFrame, date: str):
    followers_change = get_followers_change(history_df, date)
    df = get_df(date)
    if df.empty:
        return f"Nothing to update for {date}"
    if "PERFORMANCE:" in df.index:
        df = df.iloc[:df.index.get_loc("PERFORMANCE:") - 9, :]

    add_followers(df, followers_change)
    stats_df = more_stats(df, followers_change, datetime.strptime(date, "%b %d %Y").strftime("%Y-%m-%d"))

    empty = pd.Series(name="", dtype=str)
    df = df.append([empty] * 4)
    write_df(df, date)
    write_df(stats_df, date, loc=f"B{len(df)}")
    wsh = get_worksheet(date)
    wsh.format(f"C{wsh.find('Unfollowers').row}", {"textFormat": {
        "foregroundColor": {
            "red": 0.0,
            "green": 0.0,
            "blue": 1.0
        }}},)

In [ ]:
# update_insights(history_df, date)

In [ ]:
#export
def save_followers() -> str:
    df = get_df("History")
    end_time, followers = get_followers()
    df = get_updated_followers(df, followers, end_time)
    if df.empty:
        return f"No followers change, end_time '{end_time}'"
    date = df.columns[0].split("  ")[0]
    write_df(df, "History")
    save_ads_status(df)
    followers_change = get_followers_change(df, date)
    update_dashboard(date, followers_change)
    update_insights(df, date)
    return f"Wrote followers and insights for '{date}'"

In [ ]:
save_followers()

"No followers change, end_time '2020-09-22'"

### Get difference from the previous day

In [ ]:
df = get_df("History")

In [ ]:
change_df = df.diff(axis=1, periods=-1)

In [ ]:
change_df = change_df.fillna(0).astype(int)

In [ ]:
mask = (change_df != 0).any(axis=0)

In [ ]:
change_df = change_df.loc[:, mask]

In [ ]:
change_df = change_df.applymap(lambda x: f"+{x}" if x >0 else x)

In [ ]:
change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, mask].astype(str)

In [ ]:
change_df = change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
#export
def make_change(df: pd.DataFrame) -> pd.DataFrame:
    change_df = df.diff(axis=1, periods=-1)
    change_df = change_df.fillna(0).astype(int)
    zeros_mask  = (change_df != 0).any(axis=0)
    change_df = change_df.loc[:, zeros_mask]
    change_df = change_df.applymap(lambda x: f"+{x}" if x > 0 else x)
    change_df = "(" + change_df.astype(str) + ")" + " " + df.loc[:, zeros_mask].astype(str)
    return change_df.replace(["\(0\) 0", "\(0\) "], "", regex=True)

In [ ]:
#export
def save_change():
    df = get_df("History")
    change_df = make_change(df)
    write_df(change_df, "Change History")
    ads_status = get_df("Ads Status History")
    write_df(ads_status, "Change History", f"A{len(change_df)+3}", columns=False)

In [ ]:
save_change()

In [ ]:
test_change_df = make_change(pd.DataFrame({"2020-07-21": [1001, 724, 55, 2250, 1680],
                                           "2020-07-20": [1004, 708, 55, 2300, 1600],
                                           "2020-07-19": [1004, 708, 55, 2300, 1600]},
                                          index=["AR", "AU", "BD", "BR", "CA"]))
assert test_change_df.iloc[4,:].to_list() == ["(+80) 1680"]
assert test_change_df.iloc[3,:].to_list() == ["(-50) 2250"]

In [ ]:
#export
def update(event: Dict = None, context=None,):
    get_followers()
    message = save_followers()
    save_change()
    return message

In [ ]:
update()

"No followers change, end_time '2020-09-22'"

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 01_followers.ipynb.
Converted 02_ads.ipynb.
Converted index.ipynb.
